In [10]:
# Import z3
from z3 import *

#### Sudoku Puzzle Setup and Constraints
This section initializes the Sudoku puzzle solver:

- **Grid Size (N)**: Sets the size of the Sudoku grid (9x9).

- **Mappings (pos_to_cell, cell_to_pos)**: 
  - `pos_to_cell`: Maps grid positions to Bit Vectors for each cell.
  - `cell_to_pos`: Reverse mapping from Bit Vectors to grid positions.

- **Constraints**:
  - **Cell Values (val_c)**: Ensures cell values are within 1 to 9.
  - **Row Uniqueness (row_c)**: Enforces unique values in each row.
  - **Column Uniqueness (col_c)**: Enforces unique values in each column.
  - **Subgrid Uniqueness (subgrid_c)**: Ensures 3x3 subgrids have unique values.

These setup the framework and rules for the Sudoku solver, ensuring solutions follow standard Sudoku rules.

In [11]:
# Size of the sudoku grid
N = 9

# Map for position to cell
pos_to_cell = {BitVecVal(i + 1, 16): BitVec('Cell_%i' % (i + 1), 16) for i in range(N * N)}

# Map for cell to position
cell_to_pos = {v: k for k, v in pos_to_cell.items()}

# Constraints for valid cell values
val_c = [And(1 <= pos_to_cell[BitVecVal(i + 1, 16)], pos_to_cell[BitVecVal(i + 1, 16)] <= N) for i in range(N * N)]

# Constraints for distinct values in rows and columns
row_c = [Distinct([pos_to_cell[BitVecVal(i * N + j + 1, 16)] for j in range(N)]) for i in range(N)]
col_c = [Distinct([pos_to_cell[BitVecVal(i * N + j + 1, 16)] for i in range(N)]) for j in range(N)]

# Constraints for distinct values in 3x3 subgrids
subgrid_c = [Distinct([pos_to_cell[BitVecVal(subgridRow * N + subgridCol + k % 3 + N * (k // 3) + 1, 16)] for k in range(N)]) for subgridRow in range(0, N, 3) for subgridCol in range(0, N, 3)]

#### Class Overview: SudokuPropagator
In the context of computational constraint satisfaction problems, `SudokuPropagator` extends the capabilities of a base constraint propagation system (`UserPropagateBase`). This class specifically addresses the complexities involved in solving Sudoku puzzles through advanced propagation techniques.

- **Constructor Method (__init__)**:
  - Inherits from `UserPropagateBase`, aligning with object-oriented programming principles.
  - Initializes data structures: `trail` for backtracking and `lim` for maintaining backtrack boundaries.
  - Implements lambda functions to handle fixed cell assignments (`add_fixed`) and the finalization of the puzzle state (`add_final`).
  - Constructs a bidirectional mapping between cell identities and their respective positions within the Sudoku grid, aiding in efficient value retrieval and assignment.
  - Registers each cell as a variable within the constraint propagation framework.

- **Core Methods**:
  - **push**: Emphasizes the importance of state preservation in backtrack-based algorithms, storing the current trail's length.
  - **pop**: Demonstrates the backtracking mechanism, reverting state changes in response to dead-ends or conflict scenarios.
  - **fresh**: Illustrates the concept of context-sensitive instances, generating new solver instances for varied problem contexts.
  - **_fixed**: Operationalizes the assignment of definitive values to specific cells, marking a critical step in puzzle progression.
  - **_fixed_trail**: Integrates trail updates with conflict checks, underlining the dynamic nature of constraint satisfaction in Sudoku.
  - **_final**: Signifies the resolution of the puzzle, an essential component in termination conditions for constraint-based solvers.
  - **print_grid**: Provides a visualization tool for representing the current state of the Sudoku grid, crucial for both debugging and solution verification.
  - **check_conflict**: Enforces the foundational rules of Sudoku, ensuring no numerical duplications within rows, columns, and 3x3 subgrids, thereby maintaining the integrity of the puzzle's solution.

Overall, `SudokuPropagator` encapsulates the essential elements of constraint satisfaction and propagation within the realm of Sudoku puzzles, serving as an exemplary model for understanding complex problem-solving algorithms in computational theory.

In [12]:
class SudokuPropagator(UserPropagateBase):
    def __init__(self, s=None, ctx=None):
        UserPropagateBase.__init__(self, s, ctx)
        self.trail = []         # Trail for storing undo actions
        self.lim = []           # Limit for the trail stack
        self.add_fixed(lambda x, v: self._fixed(x, v))  # Handler for fixed cells
        self.add_final(lambda: self._final())           # Handler for final state

        self._fixed_le = []     # List of fixed cells and their values
        self.cell_to_pos = cell_to_pos  # Mapping from cells to positions

        for _, v in pos_to_cell.items():
            self.add(v)        # Register all variables (cells)

    def push(self):
        self.lim += [len(self.trail)]  # Push current trail length onto the limit stack

    def pop(self, n):
        # Pop 'n' elements from the limit stack and undo actions on the trail stack
        head = self.lim[len(self.lim) - n]
        while len(self.trail) > head:
            self.trail[-1]()
            self.trail.pop(-1)
        self.lim = self.lim[0:len(self.lim)-n]

    def fresh(self, new_ctx):
        # Create a fresh instance of the class for a new context
        return SudokuPropagator(ctx=new_ctx)

    def _fixed(self, cell, value):
        # Handle a cell being fixed to a value
        print("Fixed: ", cell, " := ", value)
        self._fixed_trail(cell, value, self._fixed_le)

    def _fixed_trail(self, cell, value, trail):
        # Check for conflicts and add the fixed cell to the trail
        if self.check_conflict(cell, value):
            return

        trail.append((cell, value))        
        def undo():
            trail.pop(-1)
        self.trail.append(undo)

        self.print_grid()

    def _final(self):
        # Handle the final state
        print("Final")

    def print_grid(self):
        # Initialize an empty 9x9 grid
        grid = [[0 for _ in range(N)] for _ in range(N)]

        # Fill the grid with values from fixed cells
        for cell, value in self._fixed_le:
            pos = self.cell_to_pos[cell].as_long() - 1
            row, col = pos // N, pos % N
            grid[row][col] = value.as_long()

        # Print the grid
        print("Grid:")
        for row in grid:
            print(" ".join(str(num) if num != 0 else '.' for num in row))
        print()

    def check_conflict(self, cell, value):
        # Check if a cell's value conflicts with any fixed cell

        # Extract the value and convert to an integer
        value_num = value.as_long()

        # Extract the position for the cell and convert to row and column indices
        cell_pos = self.cell_to_pos[cell].as_long()
        row, col = (cell_pos - 1) // N, (cell_pos - 1) % N

        for fixed_cell in self._fixed_le:
            other_cell, other_value = fixed_cell
            other_cell_pos = self.cell_to_pos[other_cell].as_long()
            other_row, other_col = (other_cell_pos - 1) // N, (other_cell_pos - 1) % N
            other_value_num = other_value.as_long()

            # Check for conflicts in the same row, column, or subgrid
            if value_num == other_value_num:
                # Check same row or column
                if row == other_row or col == other_col:
                    print(f"Conflict: {cell}: {value_num} conflicts with {other_cell} in the same row/column")
                    self.conflict(deps=[cell, other_cell])
                    return True

                # Check same subgrid (3x3)
                if (row // 3 == other_row // 3) and (col // 3 == other_col // 3):
                    print(f"Conflict: {cell}: {value_num} conflicts with {other_cell} in the same 3x3 subgrid")
                    self.conflict(deps=[cell, other_cell])
                    return True

        return False

#### Function Overview: solve_sudoku
This function demonstrates an application of constraint satisfaction principles for solving Sudoku puzzles. It encapsulates the process from initializing the problem space to finding a viable solution.

- **Solver Initialization**:
  - A `Solver` instance (`s`) is created, representing the core engine for constraint solving.
  - `SudokuPropagator` (`sp`) is instantiated with the solver, integrating specific Sudoku puzzle solving strategies.

- **Adding Constraints**:
  - The solver is equipped with various constraints essential for Sudoku:
    - `val_c`: Ensures all cell values are within the valid range (1 to 9).
    - `row_c`, `col_c`, `subgrid_c` (commented out): Enforce unique values in rows, columns, and 3x3 subgrids respectively.
  - Custom constraints based on the initial puzzle state are added, setting predefined values for certain cells.

- **Solving Process**:
  - The solver checks for the satisfiability (`s.check()`) of the imposed constraints.
  - If a solution exists (`sat`), the model (`m`) representing the solution is used to construct the solved puzzle grid.
  - If no solution is found, `None` is returned, indicating the puzzle's unsolvability under the given constraints.

This function is an excellent representation of applying constraint satisfaction techniques in a practical setting, showcasing the integration of problem-specific constraints and the utilization of a solver to navigate the solution space.

In [13]:
def solve_sudoku(puzzle):
    s = Solver()
    sp = SudokuPropagator(s)

    # Add constraints for the puzzle, rows, columns, and subgrids
    # As many constraints as you like
    s.add(val_c)
    # s.add(row_c)
    # s.add(col_c)
    # s.add(subgrid_c)

    for i in range(N):
        for j in range(N):
            if puzzle[i][j] != 0:
                s.add(pos_to_cell[BitVecVal(i * N + j + 1, 16)] == puzzle[i][j])

    # Solve the puzzle
    if s.check() == sat:
        m = s.model()
        return [[m.evaluate(pos_to_cell[BitVecVal(i * N + j + 1, 16)]).as_long() for j in range(N)] for i in range(N)]
    else:
        return None

In [14]:
# Example puzzle to solve
puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

In [15]:
# Solve the puzzle
solution = solve_sudoku(puzzle)

Fixed:  Cell_1  :=  5
Grid:
5 . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Fixed:  Cell_2  :=  3
Grid:
5 3 . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Fixed:  Cell_5  :=  7
Grid:
5 3 . . 7 . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Fixed:  Cell_10  :=  6
Grid:
5 3 . . 7 . . . .
6 . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Fixed:  Cell_13  :=  1
Grid:
5 3 . . 7 . . . .
6 . . 1 . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .

Fixed:  Cell_14  :=  9
Grid:
5 3 . . 7 . . 

In [16]:
# Function to print the solution
def print_sudoku(solution):
    if solution is None:
        print("No solution found.")
        return
    print("Sudoku Solution:")
    print("-" * 21)
    for i, row in enumerate(solution):
        row_str = ' | '.join(' '.join(str(cell) for cell in row[j:j+3]) for j in range(0, 9, 3))
        print(row_str)
        if i % 3 == 2:
            print("-" * 21)

In [17]:
# Print the solution
print_sudoku(solution)

Sudoku Solution:
---------------------
5 3 4 | 6 7 8 | 9 1 2
6 7 2 | 1 9 5 | 3 4 8
1 9 8 | 3 4 2 | 5 6 7
---------------------
8 5 9 | 7 6 1 | 4 2 3
4 2 6 | 8 5 3 | 7 9 1
7 1 3 | 9 2 4 | 8 5 6
---------------------
9 6 1 | 5 3 7 | 2 8 4
2 8 7 | 4 1 9 | 6 3 5
3 4 5 | 2 8 6 | 1 7 9
---------------------
